In [2]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

# Đường dẫn tới dữ liệu huấn luyện, kiểm tra và kiểm tra cuối cùng
train_dir = 'train'
val_dir = 'val'
test_dir = 'test'

# Sử dụng ImageDataGenerator để tăng cường dữ liệu
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

val_test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

val_generator = val_test_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

test_generator = val_test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

# Tải mô hình VGG16 đã được huấn luyện trước
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Đóng băng các lớp của mô hình VGG16
for layer in base_model.layers:
    layer.trainable = False

# Thêm các lớp Dense mới
x = base_model.output
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(len(train_generator.class_indices), activation='softmax')(x)

# Tạo mô hình mới
model = Model(inputs=base_model.input, outputs=predictions)

# Biên dịch mô hình
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Huấn luyện mô hình
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=val_generator,
    validation_steps=val_generator.samples // val_generator.batch_size,
    epochs=20
)

# Đánh giá mô hình trên tập kiểm tra
test_loss, test_acc = model.evaluate(test_generator, steps=test_generator.samples // test_generator.batch_size)
print('Test accuracy:', test_acc)

# Lưu mô hình
model.save('recycle_classifier_vgg16.h5')

Found 26938 images belonging to 4 classes.
Found 17646 images belonging to 4 classes.
Found 10639 images belonging to 4 classes.
Epoch 1/20
841/841 ━━━━━━━━━━━━━━━━━━━━ 3330s 4s/step - accuracy: 0.6784 - loss: 0.7736 - val_accuracy: 0.8993 - val_loss: 0.2994
Epoch 2/20
841/841 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.8438 - loss: 0.2782 - val_accuracy: 1.0000 - val_loss: 0.1052
Epoch 3/20
841/841 ━━━━━━━━━━━━━━━━━━━━ 3255s 4s/step - accuracy: 0.8688 - loss: 0.3567 - val_accuracy: 0.9372 - val_loss: 0.2070
Epoch 4/20
841/841 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.8750 - loss: 0.1186 - val_accuracy: 0.8571 - val_loss: 0.1187
Epoch 5/20
841/841 ━━━━━━━━━━━━━━━━━━━━ 3250s 4s/step - accuracy: 0.9075 - loss: 0.2564 - val_accuracy: 0.9713 - val_loss: 0.1188
Epoch 6/20
841/841 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.8125 - loss: 0.1639 - val_accuracy: 0.9286 - val_loss: 0.0937
Epoch 7/20
841/841 ━━━━━━━━━━━━━━━━━━━━ 3266s 4s/step - accuracy: 0.9293 - loss: 0.2039 - val_acc

Test accuracy: 0.9936935305595398


## Model Sửa Lại Để Nhận Diện 5 Phân Lớp Bao Gồm Cả Lớp 'None Objects'

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

# Đường dẫn tới dữ liệu huấn luyện, kiểm tra và kiểm tra cuối cùng
train_dir = 'train'
val_dir = 'val'
test_dir = 'test'

# Sử dụng ImageDataGenerator để tăng cường dữ liệu
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

val_test_datagen = ImageDataGenerator(rescale=1./255)

# Khởi tạo bộ tạo dữ liệu với thư mục train bao gồm 5 lớp (4 loại rác và "None Objects")
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

val_generator = val_test_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

test_generator = val_test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

# Tải mô hình VGG16 đã được huấn luyện trước
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Đóng băng các lớp của mô hình VGG16
for layer in base_model.layers:
    layer.trainable = False

# Thêm các lớp Dense mới
x = base_model.output
x = Flatten()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
# Cập nhật lớp đầu ra với 5 lớp (4 loại rác + "None Objects")
predictions = Dense(5, activation='softmax')(x)

# Tạo mô hình mới
model = Model(inputs=base_model.input, outputs=predictions)

# Biên dịch mô hình
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Huấn luyện mô hình
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=val_generator,
    validation_steps=val_generator.samples // val_generator.batch_size,
    epochs=20
)

# Đánh giá mô hình trên tập kiểm tra
test_loss, test_acc = model.evaluate(test_generator, steps=test_generator.samples // test_generator.batch_size)
print('Test accuracy:', test_acc)

# Lưu mô hình
model.save('recycle_classifier_vgg16_with_none.h5')